In [1]:
## PDFデータのロード
### ライブラリの読み込み
import os
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [2]:
loader = PyPDFLoader("/Users/sotaaraki/ResearchSummarizer/data/electronics-10-00407-v2.pdf")
pages = loader.load_and_split()
print(pages[0])

page_content='electronics\nArticle\nA Machine Learning Approach for Anomaly Detection in\nIndustrial Control Systems Based on Measurement Data\nSohrab Mokhtari1,*, Alireza Abbaspour2, Kang K. Yen1and Arman Sargolzaei3\n/gid00030/gid00035/gid00032/gid00030/gid00038/gid00001/gid00033/gid00042/gid00045 /gid00001\n/gid00048/gid00043/gid00031/gid00028/gid00047/gid00032/gid00046\nCitation: Mokhtari, S.; Abbaspour,\nA.; Yen, K.K.; Sargolzaei, A. A\nMachine Learning Approach for\nAnomaly Detection in Industrial\nControl Systems Based on\nMeasurement Data. Electronics 2021 ,\n10, 407. https://doi.org/10.3390/\nelectronics10040407\nAcademic Editor: Rashid Mehmood\nReceived: 1 January 2021\nAccepted: 1 February 2021\nPublished: 8 February 2021\nPublisher’s Note: MDPI stays neu-\ntral with regard to jurisdictional clai-\nms in published maps and institutio-\nnal afﬁliations.\nCopyright: © 2021 by the authors. Li-\ncensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistribute

In [3]:
pages

[Document(page_content='electronics\nArticle\nA Machine Learning Approach for Anomaly Detection in\nIndustrial Control Systems Based on Measurement Data\nSohrab Mokhtari1,*, Alireza Abbaspour2, Kang K. Yen1and Arman Sargolzaei3\n/gid00030/gid00035/gid00032/gid00030/gid00038/gid00001/gid00033/gid00042/gid00045 /gid00001\n/gid00048/gid00043/gid00031/gid00028/gid00047/gid00032/gid00046\nCitation: Mokhtari, S.; Abbaspour,\nA.; Yen, K.K.; Sargolzaei, A. A\nMachine Learning Approach for\nAnomaly Detection in Industrial\nControl Systems Based on\nMeasurement Data. Electronics 2021 ,\n10, 407. https://doi.org/10.3390/\nelectronics10040407\nAcademic Editor: Rashid Mehmood\nReceived: 1 January 2021\nAccepted: 1 February 2021\nPublished: 8 February 2021\nPublisher’s Note: MDPI stays neu-\ntral with regard to jurisdictional clai-\nms in published maps and institutio-\nnal afﬁliations.\nCopyright: © 2021 by the authors. Li-\ncensee MDPI, Basel, Switzerland.\nThis article is an open access article\n

In [6]:
os.environ["OPENAI_API_KEY"] = 'API_KEY'

In [7]:
chroma_index = Chroma.from_documents(pages, OpenAIEmbeddings())
docs = chroma_index.similarity_search("論文の概要を教えて", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

12: Electronics 2021 ,10, 407 13 of 13
8. Wang, Y.; Meng, W.; Li, W.; Li, J.; Liu, W.X.; Xiang, Y. A fog-based privacy-preserving approach for distributed signature-based
intrusion detection. J. Parallel Distrib. Comput. 2018 ,122, 26–35. [CrossRef]
9. Aloqaily, M.; Otoum, S.; Al Ridhawi, I.; Jararweh, Y. An intrusion detection system for connected vehicles in smart cities. Ad Hoc
Netw. 2019 ,90, 101842. [CrossRef]
10. Vinayakumar, R.; Alazab, M.; Soman, K.; Poornachandran, P .; Al-Nemrat, A.; Venkatraman, S. Deep learning approach for
intelligent intrusion detection system. IEEE Access 2019 ,7, 41525–41550. [CrossRef]
11. Manzoor, I.; Kumar, N. A feature reduced intrusion detection system using ANN classiﬁer. Expert Syst. Appl. 2017 ,88, 249–257.
12. Choi, W.; Joo, K.; Jo, H.J.; Park, M.C.; Lee, D.H. Voltageids: Low-level communication characteristics for automotive intrusion
detection system. IEEE Trans. Inf. Forensics Secur. 2018 ,13, 2114–2129. [CrossRef]
13. Cho, K.T.; Shin, K.G. 

---

In [9]:
## PDFデータのロード
### ライブラリの読み込み
import os
import openai
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFLoader

## Embedding
### ライブラリの読み込み
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

### Embedding & Indexing
faiss_index = FAISS.from_documents(
    pages,
    OpenAIEmbeddings()# 文章の分散表現を作成するEmbeddingモデル
    )
faiss_index.save_local("../database/faiss_index")

In [10]:
## Q&A
### ライブラリの読み込み
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

### FAISS vectorのロード
vectoreStore = FAISS.load_local("../database/faiss_index/", OpenAIEmbeddings())
## Retriever
retriever = vectoreStore.as_retriever(search_type="similarity", search_kwargs={"k":3})

### プロンプト(Q&A)
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=False)

query ="""
概要を教えてください。
"""
ans = qa({"query": query})
print("Query: \n", query)
print("Ans: \n", ans["result"])

Query: 
 
概要を教えてください。

Ans: 
 この論文では、HIL（ハードウェア・イン・ザ・ループ）シミュレータを使用した改良型ICS（工業制御システム）セキュリティデータセットを用いて、監視型侵入検知システム（MIDS）のモデルを提案しています。提案手法は、機械学習アルゴリズムを使用して、ICSのノーマルおよび異常な動作を検出することを目的としています。研究では、K最近傍法（KNN）、ランダムフォレスト（RF）、決定木分類器（DTC）の3つの教師あり分類モデルが実装され、それぞれの性能が評価されました。結果として、RFアルゴリズムが最も高い精度と効率を示したことが示されています。


In [11]:
query ="""
Introducitonではどのようなことが書いてありましたか？
"""
ans = qa({"query": query})
print("Query: \n", query)
print("Ans: \n", ans["result"])

Query: 
 
Introducitonではどのようなことが書いてありましたか？

Ans: 
 Introductionでは、異常検知の概念について説明されています。異常検知は、侵入検知、障害検知、センサーネットワークでのイベント検知など、さまざまなドメインで行われます。ICSにおいては、正常な動作からの逸脱が異常と見なされ、故障検知や侵入検知として扱われます。また、MIDS（Measurement-based Intrusion Detection System）による異常検知の性能についても述べられています。


In [13]:
def answer(query):
    ans = qa({"query": query})
    print("Query: \n", query)
    print("Ans: \n", ans["result"])

In [14]:
query ="""
    この論文は何が他と違って、秀でているのですか？
    """
answer(query)

Query: 
 
    この論文は何が他と違って、秀でているのですか？
    
Ans: 
 この論文は、計測データを使用して工業制御システム（ICS）の侵入検知を行うための機械学習モデルを開発し、故障検知の効果を評価するためにHIL（ハードウェア・イン・ループ）ベースの拡張されたICS（HAI）テストベッドのデータセットを使用するという点で他の研究とは異なります。従来の研究では、一般的に正常な活動データセットを使用して機械学習モデルをトレーニングしていましたが、この論文では制御システムの異常を検出するために制御システムの計測データを活用しています。また、HILテストベッドを使用することで、実際のシステムに損害を与えることなく、制御システムのセキュリティを向上させるためのラベル付きデータセットを生成することができます。さらに、異なる機械学習手法を比較し、提案されたデータセットに最適な学習モデルを見つけるためにランダムフォレストアルゴリズムを使用しています。


In [15]:
query ="""
    この論文で行っているランダムフォレストのモデルを組むまでの前処理と、どのようにモデルを組んでいるかを説明してください。
    なお、順序立てて説明してください。
    """
answer(query)

Query: 
 
    この論文で行っているランダムフォレストのモデルを組むまでの前処理と、どのようにモデルを組んでいるかを説明してください。
    なお、順序立てて説明してください。
    
Ans: 
 この論文では、HILベースの拡張ICSセキュリティ（HAI）のデータセットを使用して、教師あり機械学習アルゴリズムを実装しています。

まず、データセットの前処理が行われます。データセットに含まれるセンサーの測定値は、幅広い値を持っています。スケーリングされていないデータは、モデルのトレーニング手順で重大な問題を引き起こし、持続可能な学習プロセスにつながる可能性があります。そのため、MinMaxScaler関数を使用して測定データを適切な範囲にスケーリングします。

前処理のステップが完了したら、データセットは機械学習モデルのトレーニングに使用する準備ができます。この研究では、K最近傍法（KNN）、ランダムフォレスト（RF）、決定木分類器（DTC）の教師あり分類モデルがデータセットに実装されています。これらのアルゴリズムは、この特定の問題に対して効果的であるため選ばれました。アルゴリズムのパフォーマンス評価では、2つの主要な要素が考慮されます。まず、対象の出力を分類するための正確さです。次に、適合と予測のために必要な時間です。最初の要素に関しては、混同行列が計算されます。この混同行列に基づいて、正確度、適合度、F1スコア、特異度、感度などの他の指標が計算されます。2番目の要素に関しては、トレーニングデータセットの適合とテストデータセットの予測のための計算時間が取得されます。

以上の手順により、ランダムフォレストモデルが構築されます。


In [16]:
query ="""
    データセットについて、各バージョンで何が更新されたのか、わかりやすいようにそれぞれのバージョンごとに説明してください。また、データセットの取得条件について詳細に説明してください。
    """
answer(query)

Query: 
 
    データセットについて、各バージョンで何が更新されたのか、わかりやすいようにそれぞれのバージョンごとに説明してください。また、データセットの取得条件について詳細に説明してください。
    
Ans: 
 申し訳ありませんが、データセットの更新や取得条件についての具体的な情報は提供されていません。


In [17]:
query ="""
    使用しているデータについての情報を教えてください。
    """
answer(query)

Query: 
 
    使用しているデータについての情報を教えてください。
    
Ans: 
 使用しているデータは、HILベースの拡張型ICSセキュリティ（HAI）のデータセットです。このテストベッドのデータセットは、4日間にわたって毎秒59個のセンサーの測定値を収集したものです。この期間中には28の攻撃がシステムに注入されています。これらの攻撃は、プロセス制御ループ（PCL）の14の基本的な攻撃の組み合わせであり、システムの4つのポイント（設定点、プロセス変数、制御出力、制御パラメータ）に影響を与えます。これらの攻撃はステルスタイプであり、従来のNIDSでは容易に検出することができません。


In [ ]:
query ="""
    この論文の概要を教えてください。
    """
answer(query)